In [12]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

## 데이터 전처리

In [13]:
# train data
label_path = "./data/train/label/"
raw_path = "./data/train/raw/"

train_label = pd.read_csv(label_path+"1.걸음걸이/training_label.csv")
train_activity = pd.read_csv(raw_path+'train_activity.csv')
train_sleep = pd.read_csv(raw_path+'train_sleep.csv')
train_mmse = pd.read_csv(raw_path+'train_mmse.csv')

# validation -> test data
label_path = "./data/validation/label/"
raw_path = "./data/validation/raw/"

# test_label = pd.read_csv(label_path+"val_label.csv")
test_activity = pd.read_csv(raw_path+'val_activity.csv')
test_sleep = pd.read_csv(raw_path+'val_sleep.csv')
test_mmse = pd.read_csv(raw_path+'val_mmse.csv')

In [14]:
print("train_label:", train_label.shape)
print("train_activity:", train_activity.shape)
print("train_sleep:", train_sleep.shape)
print("train_mmse:", train_mmse.shape)
print("---"*10)
# print("test_label:", test_label.shape)
print("test_activity:", test_activity.shape)
print("test_sleep:", test_sleep.shape)
print("test_mmse:", test_mmse.shape)

train_label: (141, 2)
train_activity: (9705, 31)
train_sleep: (9705, 36)
train_mmse: (141, 38)
------------------------------
test_activity: (2478, 31)
test_sleep: (2478, 36)
test_mmse: (33, 38)


In [16]:
train_label['DIAG_NM'].count()

np.int64(141)

In [10]:
train_activity['activity_day_start'].max()

'2021-02-17T04:00:00+09:00'

In [5]:
activity = pd.concat([train_activity, test_activity])
sleep = pd.concat([train_sleep, test_sleep])
mmse = pd.concat([train_mmse, test_mmse])

print("activity:", activity.shape)
print("sleep:", sleep.shape)
print("mmse:", mmse.shape)

activity: (12183, 31)
sleep: (12183, 36)
mmse: (174, 38)


In [6]:
train_label.rename(columns={"SAMPLE_EMAIL": "EMAIL"}, inplace=True)
test_label.rename(columns={"SAMPLE_EMAIL": "EMAIL"}, inplace=True)
mmse.rename(columns={"SAMPLE_EMAIL": "EMAIL"}, inplace=True)

### 불필요한 열 제거

In [7]:
# '...'로 채워진 원본 열을 제거하고 수치 데이터가 있는 열만 남김
activity.drop(columns=['activity_class_5min', 'activity_met_1min', 
                       'activity_day_end',], axis=1, inplace=True)

sleep.drop(columns=['sleep_hr_5min', 'sleep_hypnogram_5min', 
                    'sleep_rmssd_5min'], axis=1, inplace=True)

### 컬럼 rename, datetime 형 변환

In [8]:
activity.rename(columns={'CONVERT(activity_class_5min USING utf8)':'activity_met_class_5min',
                         'CONVERT(activity_met_1min USING utf8)': 'activity_met_1min',
                        }, inplace=True)
sleep.rename(columns={'CONVERT(sleep_hr_5min USING utf8)':'sleep_hr_5min)',
                      'CONVERT(sleep_hypnogram_5min USING utf8)':'sleep_hypnogram_5min',
                      'CONVERT(sleep_rmssd_5min USING utf8)':'sleep_rmssd_5min'
                      }, inplace=True)

In [9]:
print(f"Activity Data Shape: {activity.shape}")
print(f"Sleep Data Shape: {sleep.shape}")

Activity Data Shape: (12183, 28)
Sleep Data Shape: (12183, 33)


In [10]:
activity.rename(columns={"activity_day_start": "activity_date"}, inplace=True)

In [11]:
activity['activity_date'] = pd.to_datetime(activity['activity_date']).dt.tz_localize(None).dt.normalize()

sleep['sleep_bedtime_end'] = pd.to_datetime(sleep['sleep_bedtime_end']).dt.tz_localize(None)
sleep['sleep_bedtime_start'] = pd.to_datetime(sleep['sleep_bedtime_start']).dt.tz_localize(None)

### 컬럼 정렬 및 log 컬럼 분리

In [12]:
activity_sorted = ['EMAIL', 'activity_date', 'activity_non_wear', 
                   'activity_total', 'activity_cal_active',
                   'activity_cal_total', 'activity_daily_movement', 'activity_steps',
                   'activity_inactivity_alerts', 'activity_rest', 'activity_inactive', 'activity_low', 'activity_medium', 'activity_high', 
                   'activity_met_min_inactive','activity_met_min_low','activity_met_min_medium', 'activity_met_min_high','activity_average_met', 
                   'activity_score', 'activity_score_meet_daily_targets',
                   'activity_score_move_every_hour', 'activity_score_recovery_time',
                   'activity_score_stay_active', 'activity_score_training_frequency', 'activity_score_training_volume']
activity_log = ['EMAIL', 'activity_date', 'activity_met_class_5min', 'activity_met_1min']

sleep_sorted = ['EMAIL', 'sleep_bedtime_start', 'sleep_bedtime_end',
                'sleep_period_id','sleep_duration', 'sleep_total','sleep_awake', 'sleep_rem', 'sleep_light', 'sleep_deep', 
                'sleep_efficiency',  'sleep_midpoint_at_delta','sleep_midpoint_time', 
                'sleep_onset_latency','sleep_is_longest',
                'sleep_breath_average','sleep_hr_average', 'sleep_hr_lowest', 'sleep_restless',  'sleep_rmssd',
                'sleep_score', 'sleep_score_alignment', 'sleep_score_deep', 'sleep_score_disturbances',
                'sleep_score_efficiency', 'sleep_score_latency', 'sleep_score_rem', 'sleep_score_total', 'sleep_temperature_delta','sleep_temperature_deviation']
sleep_log = ['EMAIL', 'sleep_bedtime_start', 'sleep_hr_5min)', 'sleep_hypnogram_5min', 'sleep_rmssd_5min']

print("activity columns:", len(activity_sorted))
print("activity log columns:", len(activity_log))
print("sleep columns:", len(sleep_sorted))
print("sleep log columns:", len(sleep_log))

activity columns: 26
activity log columns: 4
sleep columns: 30
sleep log columns: 5


In [13]:
activity_log = activity[activity_log]
activity = activity[activity_sorted]
sleep_log = sleep[sleep_log]
sleep = sleep[sleep_sorted]

### 데이터 저장

In [ ]:
folder="data/processed"
os.makedirs(folder, exist_ok=True)

In [14]:
activity.to_csv(folder+"/activity.csv", index=False)
activity_log.to_csv(folder+"/log/activity_log.csv", index=False)
sleep.to_csv(folder+"/sleep.csv", index=False)
sleep_log.to_csv(folder+"/log/sleep_log.csv", index=False)

In [15]:
activity = pd.read_csv(folder+"/activity.csv", parse_dates=['activity_date'])
activity_log = pd.read_csv(folder+"/log/activity_log.csv", parse_dates=['activity_date'])
sleep = pd.read_csv(folder+"/sleep.csv", parse_dates=['sleep_bedtime_start'])
sleep_log = pd.read_csv(folder+"/log/sleep_log.csv", parse_dates=['sleep_bedtime_start'])

In [16]:
train_label.to_csv(folder+"/train_label.csv", index=False)
test_label.to_csv(folder+"/test_label.csv", index=False)

In [ ]:
## Activity
activity_log = pd.read_csv(folder+"/log/activity_log.csv", parse_dates=['activity_date'])
activity_log.info()
### MET Class 5min
splited_log = activity_log['activity_met_class_5min'].str.split('/').apply(lambda x: pd.Series(x))
splited_log = pd.concat([activity_log, splited_log], axis=1).drop(['activity_met_class_5min', 'activity_met_1min', 288], axis=1)
stack_log = splited_log.set_index(['EMAIL', 'activity_date']).stack()
met_class = stack_log.reset_index().rename(columns={'level_2': 'time_5min_interval',
                                                  0: 'met_class'})
met_class.head()
met_class.info()
empty_idx = met_class[met_class['met_class']==''].index
met_class = met_class.drop(empty_idx, axis=0)
met_class['met_class'] = met_class['met_class'].astype(int)
met_class
met_class.to_csv('data/processed/log/met_class.csv', index=False)
### MET 1min
splited_log = activity_log['activity_met_1min'].str.split('/').apply(lambda x: pd.Series(x))
met_1min = pd.concat([activity_log, splited_log], axis=1).drop(['activity_met_class_5min', 'activity_met_1min', 1440], axis=1)
met_1min = met_1min.astype(dtype='float',errors='ignore')
met_stacked = met_1min.set_index(['EMAIL', 'activity_date']).stack()
metlog = met_stacked.reset_index().rename(columns={'level_2': 'minutes_time',
                                                  0: 'met'})
metlog
metlog.info()
metlog.to_csv('data/processed/log/met_log.csv', index=False)
## Sleep
sleep_log = pd.read_csv(folder+"/log/sleep_log.csv", parse_dates=['sleep_bedtime_start'])
met_class.to_csv('data/processed/log/met_class.csv', index=False)

## Activity log Parsing

In [ ]:
activity_log = pd.read_csv(folder+"/log/activity_log.csv", parse_dates=['activity_date'])

In [ ]:
activity_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12183 entries, 0 to 12182
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   EMAIL                    12183 non-null  object        
 1   activity_date            12183 non-null  datetime64[ns]
 2   activity_met_class_5min  12183 non-null  object        
 3   activity_met_1min        12183 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 380.8+ KB


### MET Class 5min

In [ ]:
splited_log = activity_log['activity_met_class_5min'].str.split('/').apply(lambda x: pd.Series(x))
splited_log = pd.concat([activity_log, splited_log], axis=1).drop(['activity_met_class_5min', 'activity_met_1min', 288], axis=1)

In [ ]:
stack_log = splited_log.set_index(['EMAIL', 'activity_date']).stack()
met_class = stack_log.reset_index().rename(columns={'level_2': 'time_5min_interval',
                                                  0: 'met_class'})

In [ ]:
met_class.head()

,EMAIL,activity_date,time_5min_interval,met_class
0,nia+279@rowan.kr,2020-10-19,0,1
1,nia+279@rowan.kr,2020-10-19,1,1
2,nia+279@rowan.kr,2020-10-19,2,2
3,nia+279@rowan.kr,2020-10-19,3,2
4,nia+279@rowan.kr,2020-10-19,4,2


In [ ]:
met_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3492255 entries, 0 to 3492254
Data columns (total 4 columns):
 #   Column              Dtype         
---  ------              -----         
 0   EMAIL               object        
 1   activity_date       datetime64[ns]
 2   time_5min_interval  int64         
 3   met_class           object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 106.6+ MB


In [ ]:
empty_idx = met_class[met_class['met_class']==''].index
met_class = met_class.drop(empty_idx, axis=0)
met_class['met_class'] = met_class['met_class'].astype(int)

In [ ]:
met_class

,EMAIL,activity_date,time_5min_interval,met_class
0,nia+279@rowan.kr,2020-10-19,0,1
1,nia+279@rowan.kr,2020-10-19,1,1
2,nia+279@rowan.kr,2020-10-19,2,2
3,nia+279@rowan.kr,2020-10-19,3,2
4,nia+279@rowan.kr,2020-10-19,4,2
...,...,...,...,...
3492250,nia+280@rowan.kr,2021-02-17,283,1
3492251,nia+280@rowan.kr,2021-02-17,284,1
3492252,nia+280@rowan.kr,2021-02-17,285,1
3492253,nia+280@rowan.kr,2021-02-17,286,1


In [ ]:
log_folder="data/processed/log"
os.makedirs(log_folder, exist_ok=True)
met_class.to_csv(log_folder+"/met_class.csv", index=False)

### MET 1min

In [ ]:
splited_log = activity_log['activity_met_1min'].str.split('/').apply(lambda x: pd.Series(x))
met_1min = pd.concat([activity_log, splited_log], axis=1).drop(['activity_met_class_5min', 'activity_met_1min', 1440], axis=1)

In [ ]:
met_1min = met_1min.astype(dtype='float',errors='ignore')

In [ ]:
met_stacked = met_1min.set_index(['EMAIL', 'activity_date']).stack()
metlog = met_stacked.reset_index().rename(columns={'level_2': 'minutes_time',
                                                  0: 'met'})

In [ ]:
metlog

,EMAIL,activity_date,minutes_time,met
0,nia+279@rowan.kr,2020-10-19,0,1.2
1,nia+279@rowan.kr,2020-10-19,1,0.9
2,nia+279@rowan.kr,2020-10-19,2,1.0
3,nia+279@rowan.kr,2020-10-19,3,0.9
4,nia+279@rowan.kr,2020-10-19,4,0.9
...,...,...,...,...
17543515,nia+280@rowan.kr,2021-02-17,1435,0.9
17543516,nia+280@rowan.kr,2021-02-17,1436,0.9
17543517,nia+280@rowan.kr,2021-02-17,1437,0.9
17543518,nia+280@rowan.kr,2021-02-17,1438,0.9


In [ ]:
metlog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17543520 entries, 0 to 17543519
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   EMAIL          object        
 1   activity_date  datetime64[ns]
 2   minutes_time   int64         
 3   met            float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 535.4+ MB


In [ ]:
metlog.to_csv(log_folder+"/met_log.csv", index=False)

## Sleep log Parsing

In [ ]:
sleep_log = pd.read_csv(folder+"/log/sleep_log.csv", parse_dates=['sleep_bedtime_start'])

In [ ]:
# sleep_log.to_csv(log_folder+"/sleep_log.csv", index=False)